#### * _We used Pytorch for our CNN models_ *

# __Why Detect Lane Markings__
For our project, we are interested in working on lane detection for autonomous vehicles using lane markings painted on the roads. The task of lane detection is still vital to the success of autonomous vehicles because it is pertinent that an autonomous vehicle can exactly recognize which lane it is in and where it needs to go. If it is not able to do that, the vehicles might be going in the middle of two lanes which can be dangerous for other vehicles in the vicinity. Also, lane detection is inevitable to perform maneuvers like lane changes and also for predicting other vehicles’ behavior.

# __Build a lane extractor from an image__
In this project, we tried to train a Convolutional Neural Network (CNN) to extract lane markings as features from given images of roads with lanes. 

### __Dataset Description__
We plan on using the __CULane Dataset__ (Chinese University Lane Dataset) which contains 88,880 images for the training set, 9,675 images for the validation set and 34,680 images for the test set. Furthermore, the test set is divided into one normal and 8 challenging categories that tests the network on harder images. The dataset is desirable because it contains images from a variety of settings like crowded, night, no line, shadow, curve, etc. Such diversity in the dataset is desirable for training the model.	

### __Modelling Pipeline__
  1. Extract the images from the dataset stored on Drive
  2. Organize the images into raw images and labelled value
  3. Train the Model
  4. Get a reasonable validation accuracy
  5. Test the Model on unseen images 

In [17]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
CODE = 'My Drive/cs231n/Project/LaneNet'       # path to our code on Drive
DATASET = 'My Drive/cs231n/Project/dataset/trainset'
EXPERIMENT = 'My Drive/cs231n/Project/LaneNet/config'
MODEL_SAVE = 'My Drive/cs231n/Project'

CODE_PATH = join(ROOT, CODE)
DATASET_PATH = join(ROOT, DATASET)
EXPERIMENT_CONFIG = join(ROOT, EXPERIMENT)
MODEL_SAVE = join(ROOT, MODEL_SAVE)

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('{}'.format(CODE_PATH))



Mounted at /content/drive


## Some Imports


In [0]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

### Let's visualize some of the images we have
We have a contiguous set of 20 images frame-by-frame for each scene, with the 20th frame labelled in our dataset. <br>
Let's visualize these images to understand what each scene information contains:


In [4]:
# Obtaining a list of paths for each image
"""
FIXED_PATH = DATASET_PATH + '/clips/0313-1/60/'
items = os.listdir(FIXED_PATH)
n_images = len(items)
plt_idx = 0
fig = plt.figure()
n_cols = 20
for image in items: 
  plt_idx += 1
  full_path = FIXED_PATH + image
  a = fig.add_subplot(n_cols, np.ceil(n_images/float(n_cols)), plt_idx)
  img = plt.imread(full_path)
  #plt.imshow(img.astype('uint8'))
fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
plt.show()
"""

"\nFIXED_PATH = DATASET_PATH + '/clips/0313-1/60/'\nitems = os.listdir(FIXED_PATH)\nn_images = len(items)\nplt_idx = 0\nfig = plt.figure()\nn_cols = 20\nfor image in items: \n  plt_idx += 1\n  full_path = FIXED_PATH + image\n  a = fig.add_subplot(n_cols, np.ceil(n_images/float(n_cols)), plt_idx)\n  img = plt.imread(full_path)\n  #plt.imshow(img.astype('uint8'))\nfig.set_size_inches(np.array(fig.get_size_inches()) * n_images)\nplt.show()\n"

# __1. Extract the images from the dataset stored on Drive__ 

We have already done so in our Operational.ipynb notebook
<br>

## Some imports for the modules

In [0]:
import argparse
import json
import os
import shutil
import time

import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

#from config import *
import data
from model import SCNN
from utils.tensorboard import TensorBoard
from utils.transforms import *
from utils.lr_scheduler import PolyLR

## Loading model configurations

In [25]:
# Location of our configuration for training
exp_dir = EXPERIMENT_CONFIG + '/cfg.json'
exp_name = '0'
save_dir = MODEL_SAVE + '/' + exp_name
with open(exp_dir) as f:
  exp_cfg = json.load(f)
print(exp_cfg)

# Extracting important configurations
resize_shape = tuple(exp_cfg['dataset']['resize_shape'])

device = torch.device(exp_cfg['device'])

######################## COMMAND CENTER LATER###############
#tensorboard = TensorBoard(exp_dir) # Command center Later
############################################################

{'device': 'cuda:0', 'MAX_EPOCHES': 10, 'dataset': {'dataset_name': 'Tusimple', 'batch_size': 32, 'resize_shape': [512, 288]}, 'optim': {'lr': 0.15, 'momentum': 0.9, 'weight_decay': 0.0001, 'nesterov': True}, 'lr_scheduler': {'warmup': 20, 'max_iter': 1500, 'min_lrs': 1e-10}, 'model': {'scale_exist': 0.07}}


# __2. Organize the images into raw images and labelled value__

Here, we will generate labelled data for our train, validation and test splits. We will generate labelled data as bindary masks since the original dataset doesn't come preloaded with this information.
<br>
Moreover, instead of loading data manually, we decided on using Pytorch to create a custom Dataset and DataLoader
<br> 
More details on the DataLoader setup can be found in `LaneNet/Data/Tusimple.py`


### Data Pre-processing
We perform three data augmentation steps to our images:
<br>
1. Resizing images to make them smaller
2. Rotation by 2 degrees
3. Normalizing data by given mean and standard deviation


In [0]:
# Training data
# Predefined mean and std- deviation
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

transform_train = Compose(Resize(resize_shape), Rotation(2), ToTensor(),
                          Normalize(mean=mean, std=std))

### Now we perform data loading operations on our custom defined Dataset Object `namely Tusimple`
Basically, this is where we create our __train, validation and test splits__ on our dataset

In [9]:
dataset_name = exp_cfg['dataset']['dataset_name']
Dataset_Type = getattr(data, dataset_name)

# Training Data
train_dataset = Dataset_Type("", "train", transform_train)
train_loader = DataLoader(train_dataset, batch_size=exp_cfg['dataset']['batch_size'], shuffle=True, collate_fn=train_dataset.collate, num_workers=8)

# Validation Data
transform_val_img = Resize(resize_shape)
transform_val_x = Compose(ToTensor(), Normalize(mean=mean, std=std))
transform_val = Compose(transform_val_img, transform_val_x)
val_dataset = Dataset_Type("", "val", transform_val)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=val_dataset.collate, num_workers=4)

Label is going to get generated into dir: seg_label ...
/content/drive/My Drive/cs231n/Project/dataset/trainset/seg_label
train set is done
val set is done
test set is done
Label is going to get generated into dir: seg_label ...
/content/drive/My Drive/cs231n/Project/dataset/trainset/seg_label
train set is done
val set is done
test set is done


## Initializing our Neural Network Model


In [10]:
net = SCNN(resize_shape, pretrained=True)
net = net.to(device)
net = torch.nn.DataParallel(net)

optimizer = optim.SGD(net.parameters(), **exp_cfg['optim'])
lr_scheduler = PolyLR(optimizer, 0.9, **exp_cfg['lr_scheduler'])
best_val_loss = 1e6

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth


## Defining our train function

In [0]:
def train(epoch):
    print("Train Epoch: {}".format(epoch))
    net.train()
    train_loss = 0
    train_loss_seg = 0
    train_loss_exist = 0
    progressbar = tqdm(range(len(train_loader)))

    for batch_idx, sample in enumerate(train_loader):
        img = sample['img'].to(device)
        segLabel = sample['segLabel'].to(device)
        exist = sample['exist'].to(device)

        optimizer.zero_grad()
        seg_pred, exist_pred, loss_seg, loss_exist, loss = net(img, segLabel, exist)
        if isinstance(net, torch.nn.DataParallel):
            loss_seg = loss_seg.sum()
            loss_exist = loss_exist.sum()
            loss = loss.sum()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        iter_idx = epoch * len(train_loader) + batch_idx
        train_loss = loss.item()
        train_loss_seg = loss_seg.item()
        train_loss_exist = loss_exist.item()
        progressbar.set_description("batch loss: {:.3f}".format(loss.item()))
        progressbar.update(1)

        lr = optimizer.param_groups[0]['lr']
        
        # Command center later
        #tensorboard.scalar_summary(exp_name + "/train_loss", train_loss, iter_idx)
        #tensorboard.scalar_summary(exp_name + "/train_loss_seg", train_loss_seg, iter_idx)
        #tensorboard.scalar_summary(exp_name + "/train_loss_exist", train_loss_exist, iter_idx)
        #tensorboard.scalar_summary(exp_name + "/learning_rate", lr, iter_idx)

    progressbar.close()
    #tensorboard.writer.flush()

    if epoch % 1 == 0:
        save_dict = {
            "epoch": epoch,
            "net": net.module.state_dict() if isinstance(net, torch.nn.DataParallel) else net.state_dict(),
            "optim": optimizer.state_dict(),
            "lr_scheduler": lr_scheduler.state_dict(),
            "best_val_loss": best_val_loss
        }
        
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        save_name = os.path.join(save_dir, exp_name + '.pth')
        torch.save(save_dict, save_name)
        print("model is saved: {}".format(save_name))

    print("------------------------\n")

## Defining our validation function




In [0]:
def val(epoch):
    global best_val_loss

    print("Val Epoch: {}".format(epoch))

    net.eval()
    val_loss = 0
    val_loss_seg = 0
    val_loss_exist = 0
    progressbar = tqdm(range(len(val_loader)))

    with torch.no_grad():
        for batch_idx, sample in enumerate(val_loader):
            img = sample['img'].to(device)
            segLabel = sample['segLabel'].to(device)
            exist = sample['exist'].to(device)

            seg_pred, exist_pred, loss_seg, loss_exist, loss = net(img, segLabel, exist)
            if isinstance(net, torch.nn.DataParallel):
                loss_seg = loss_seg.sum()
                loss_exist = loss_exist.sum()
                loss = loss.sum()

            # visualize validation every 5 frame, 50 frames in all
            gap_num = 5
            if batch_idx%gap_num == 0 and batch_idx < 50 * gap_num:
                origin_imgs = []
                seg_pred = seg_pred.detach().cpu().numpy()
                exist_pred = exist_pred.detach().cpu().numpy()

                for b in range(len(img)):
                    img_name = sample['img_name'][b]
                    img = cv2.imread(img_name)
                    img = transform_val_img({'img': img})['img']

                    lane_img = np.zeros_like(img)
                    color = np.array([[255, 125, 0], [0, 255, 0], [0, 0, 255], [0, 255, 255]], dtype='uint8')

                    coord_mask = np.argmax(seg_pred[b], axis=0)
                    for i in range(0, 4):
                        if exist_pred[b, i] > 0.5:
                            lane_img[coord_mask==(i+1)] = color[i]
                    img = cv2.addWeighted(src1=lane_img, alpha=0.8, src2=img, beta=1., gamma=0.)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    lane_img = cv2.cvtColor(lane_img, cv2.COLOR_BGR2RGB)
                    cv2.putText(lane_img, "{}".format([1 if exist_pred[b, i]>0.5 else 0 for i in range(4)]), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 255, 255), 2)
                    origin_imgs.append(img)
                    origin_imgs.append(lane_img)
                #tensorboard.image_summary("img_{}".format(batch_idx), origin_imgs, epoch)

            val_loss += loss.item()
            val_loss_seg += loss_seg.item()
            val_loss_exist += loss_exist.item()

            progressbar.set_description("batch loss: {:.3f}".format(loss.item()))
            progressbar.update(1)

    progressbar.close()
    iter_idx = (epoch + 1) * len(train_loader)  # keep align with training process iter_idx
    #tensorboard.scalar_summary("val_loss", val_loss, iter_idx)
    #tensorboard.scalar_summary("val_loss_seg", val_loss_seg, iter_idx)
    #tensorboard.scalar_summary("val_loss_exist", val_loss_exist, iter_idx)
    #tensorboard.writer.flush()

    print("------------------------\n")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        #if not os.path.exists(save_dir):
        #  os.makedirs(save_dir)
        save_name = os.path.join(save_dir, exp_name + '.pth')
        copy_name = os.path.join(save_dir, exp_name + '_best.pth')
        shutil.copyfile(save_name, copy_name)

# __3. Train our Model__

Finally, we get to train our model on the images

In [33]:
# Let's start training
start_epoch = 0

exp_cfg['MAX_EPOCHES'] = int(np.ceil(exp_cfg['lr_scheduler']['max_iter'] / len(train_loader)))
for epoch in range(start_epoch, exp_cfg['MAX_EPOCHES']):
    train(epoch)
    if epoch % 1 == 0:
        print("\nValidation For Experiment: ", exp_dir)
        print(time.strftime('%H:%M:%S', time.localtime()))
        val(epoch)

  0%|          | 0/1 [00:00<?, ?it/s]

Train Epoch: 0


  0%|          | 0/1 [00:00<?, ?it/s]

model is saved: /content/drive/My Drive/cs231n/Project/0/0.pth
------------------------


Validation For Experiment:  /content/drive/My Drive/cs231n/Project/LaneNet/config/cfg.json
09:56:12
Val Epoch: 0


batch loss: 0.870: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


...

  0%|          | 0/1 [00:00<?, ?it/s]

model is saved: /content/drive/My Drive/cs231n/Project/0/0.pth
------------------------


Validation For Experiment:  /content/drive/My Drive/cs231n/Project/LaneNet/config/cfg.json
10:17:15
Val Epoch: 306


  0%|          | 0/1 [00:00<?, ?it/s]

------------------------

Train Epoch: 307


KeyboardInterrupt: ignored